In [1]:
!pip install pysam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.1 MB 8.5 MB/s 


In [ ]:
from pysam import FastaFile
from itertools import product

sequences_dict = {}

for i in range(1, 5):
  name = f'bacterial{i}'
  sequences_object = FastaFile(f'data/{name}.fasta')
  sequences_dict[name] = sequences_object.fetch(sequences_object.references[0])

for i in range(1, 5):
  name = f'mamalian{i}'
  sequences_object = FastaFile(f'data/{name}.fasta')
  sequences_dict[name] = sequences_object.fetch(sequences_object.references[0])

print(sequences_dict)

In [4]:
start_codons = ["ATG"]
stop_codons = ["TAG", "TGA", "TAA"]
reverse = {"A": "T", "C": "G", "T": "A", "G": "C"}

l = ["A", "T", "G", "C"]

all_codones = [''.join(i) for i in product(l, repeat = 3)]
all_dicodones = [''.join(i) for i in product(all_codones, repeat = 2)]

In [5]:
def get_all_frames(seq):
  temp_frame_list = []  
  frames = []
  for j in range (3):
    temp_frame_list.append([seq[i:i + 3] for i in range(j, len(seq), 3)])

  reverseCompl = []
  for i in range(len(seq)):
    reverseCompl.append(reverse[seq[-i - 1]])
  
  seq = ''.join(reverseCompl)
  for j in range (3):
    temp_frame_list.append([seq[i:i + 3] for i in range(j, len(seq), 3)])
    
  return temp_frame_list

In [6]:
def get_orfs(frames):
  allOrfs = []
  o = []
  for i in range(0, len(frames)):
    start = 0
    while start < len(frames[i]):
      if frames[i][start] in start_codons:
        for stop in range(start + 1, len(frames[i])):
          if frames[i][stop] in stop_codons:
            allOrfs.append(frames[i][start:stop])
            start=stop+1
            break
      start += 1
  return allOrfs

In [7]:
#tik ilgesni nei 100 ir sujungiamos sekos i viena
def get_longer_than_100_in_full_sequence_and_count(allOrfs):
  temp_list = []
  count_of_elements = 0
  one_sequence_for_freq = [] # pridedams # simbolis gale sekos, lengviau ieskant dikodonu daznio
  for orf in allOrfs:
    if (len(orf) * 3 >= 100):
      temp_list.append(orf)
      count_of_elements += len(orf)
      one_sequence_for_freq.extend(orf)
      one_sequence_for_freq.append('#')
  return (one_sequence_for_freq, count_of_elements)

In [8]:
#daznio ieskojimas
def calculate_frequency (list_of_combinations, sequence, count, find_codones = True):
  freq_dict = {k : 0 for k in list_of_combinations}

  for key in sequence: # jei ieskome kodonu daznio
    if (key in freq_dict):
      freq_dict[key] += 1
  else: # jei ieskome dikodonu daznio
    for i, key in enumerate(sequence):
      if (i + 1 >= len(sequence)): break

      next_key = sequence[i + 1]
      temp = key + next_key
      if (temp in freq_dict):
        freq_dict[temp] += 1

  for key in freq_dict.keys():
    freq_dict[key] /= count
  return freq_dict

In [9]:
def get_frequencies_in_sequence(seq):
  frames = get_all_frames(seq)
  orfs = get_orfs(frames)
  sequence_for_freq, count = get_longer_than_100_in_full_sequence_and_count(orfs)

  codone_freq = calculate_frequency(all_codones, sequence_for_freq, count, True)
  dicodone_freq = calculate_frequency(all_dicodones, sequence_for_freq, count, False)
  return (codone_freq, dicodone_freq)

In [10]:
def get_distance(seq1, seq2):
  distances = [abs(seq1[key] - seq2[key]) for key in seq1.keys()]
  return (sum(distances) / len(seq1)) * 100

In [12]:
#calculate all frequancies
freq_dict = {}
for key in sequences_dict.keys():
  freq_dict[key] = get_frequencies_in_sequence(sequences_dict[key])

In [18]:
codones_matrix = [[0 for y in range(9)] for x in range(8)]
dicodones_matrix = [[0 for y in range(9)] for x in range(8)]
i = 0
for key in freq_dict.keys():
  codones_matrix[i][0] = key
  dicodones_matrix[i][0] = key
  j = 1
  for key_other in freq_dict.keys():
    codones_matrix[i][j] = get_distance(freq_dict[key][0], freq_dict[key_other][0])
    dicodones_matrix[i][j] = get_distance(freq_dict[key][1], freq_dict[key_other][1])
    j += 1
  i += 1

In [19]:
def print_matrix(matrix):
  for i in range(8):
    s = ""
    for j in range(9):
      s += f'{matrix[i][j]} '
    print(s)

In [21]:
print_matrix(codones_matrix)

bacterial1 0.0 0.6989632554780053 0.3547384860890319 0.6126965680705813 0.4359234552562748 0.8163990274537511 0.5674011865940007 1.1311008788747685 
bacterial2 0.6989632554780053 0.0 0.5424330178370035 1.1023085995450186 0.6173478518905082 0.4307264458883532 0.7325053870089067 0.6237052853304594 
bacterial3 0.3547384860890319 0.5424330178370035 0.0 0.7337364339627372 0.33324230154284223 0.6556224454899742 0.5228894870452402 0.9776627284727365 
bacterial4 0.6126965680705813 1.1023085995450186 0.7337364339627372 0.0 0.7883026821335191 1.2126682861526006 0.7572652568035231 1.4648329172619248 
mamalian1 0.4359234552562748 0.6173478518905082 0.33324230154284223 0.7883026821335191 0.0 0.6740315551655435 0.5815370189260377 1.0295936242112151 
mamalian2 0.8163990274537511 0.4307264458883532 0.6556224454899742 1.2126682861526006 0.6740315551655435 0.0 0.8484656338032328 0.396571905060033 
mamalian3 0.5674011865940007 0.7325053870089067 0.5228894870452402 0.7572652568035231 0.5815370189260377 0.

In [22]:
print_matrix(dicodones_matrix)

bacterial1 0.0 0.02132067520447761 0.016124727783787787 0.021475600225783764 0.017914270332295285 0.023078682461597708 0.01943710807142197 0.028313825956029604 
bacterial2 0.02132067520447761 0.0 0.01872300299512442 0.027611338230720952 0.01981375979649642 0.016193482952650627 0.02107270377852177 0.019232642903248977 
bacterial3 0.016124727783787787 0.01872300299512442 0.0 0.022413145916024863 0.016030711974943015 0.019624222150055517 0.0192118307905325 0.024771008820516534 
bacterial4 0.021475600225783764 0.027611338230720952 0.022413145916024863 0.0 0.02258079957666274 0.028510757913720722 0.02234927018905029 0.03265524238788268 
mamalian1 0.017914270332295285 0.01981375979649642 0.016030711974943015 0.02258079957666274 0.0 0.01953989781245677 0.019496219343472693 0.02532687735564707 
mamalian2 0.023078682461597708 0.016193482952650627 0.019624222150055517 0.028510757913720722 0.01953989781245677 0.0 0.022643458381794785 0.015823370692033812 
mamalian3 0.01943710807142197 0.021072703